In [1]:
cd("/Users/xiaoquer/Library/Mobile Documents/com~apple~CloudDocs/Documents/Work/effective-spinfoam/code/spinfoam-critical-pts")

In [2]:
using LinearAlgebra, Combinatorics
using Printf
using Dates

include("GeometryTypes.jl")          # defines GeometryDataset, GeometryCollection
#include("load_geometry_data.jl")        # low-level parsers FIRST
#include("GeometryDataLoader.jl")        # defines GeometryDataset
# --- include all modules ---
include("SimplexGeometry.jl")
include("SpinAlgebra.jl")
include("volume.jl")
include("TetraNormals.jl")
include("DihedralAngles.jl")
include("LorentzGroup.jl")
include("ThreeDTetra.jl")
include("Su2Su11FromBivector.jl")
include("XiFromSU.jl")
include("FaceNormals3D.jl")
include("KappaFromNormals.jl")
include("GeometryConsistency.jl")
include("GeometryPipeline.jl")
include("KappaOrientation.jl")
include("FourSimplexConnectivity.jl")
include("FaceXiMatching.jl")
include("FaceMatchingChecks.jl")
include("GaugeFixing.jl")
include("CriticalPoints.jl")
include("defineVariables.jl")


using .GeometryTypes: GeometryDataset, GeometryCollection
using .GeometryPipeline: run_geometry_pipeline
using .GeometryConsistency: check_sl2c_parallel_transport, check_so13_parallel_transport, check_closure_bivectors
using .KappaOrientation: fix_kappa_signs!
using .FourSimplexConnectivity: build_global_connectivity
using .FaceXiMatching: run_face_xi_matching
using .FaceMatchingChecks: check_all
using .GaugeFixingSU
using .CriticalPoints: compute_critical_data
using .DefineVariables: run_define_variables

In [3]:
bdypoints_all = [
    [
        [0.0, 0.0, 0.0, 0.0],
        [0.0, 0.0, 1.0, 0.0],
        [0.0, 1.0, 1.0, 0.0],
        [0.5, 1.0, 1.0, 0.0],
        [0.5, 1.0, 1.0, 1.0]
    ]]

1-element Vector{Vector{Vector{Float64}}}:
 [[0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 1.0, 0.0], [0.0, 1.0, 1.0, 0.0], [0.5, 1.0, 1.0, 0.0], [0.5, 1.0, 1.0, 1.0]]

In [4]:
geom = nothing
datasets = GeometryDataset[]
ns = length(bdypoints_all)

base_folder = "geom_output/new-data/"
mkpath(base_folder)

for s in 1:ns
    bdypoints = bdypoints_all[s]
    println("Boundary points for simplex $s:")

    ds = run_geometry_pipeline(bdypoints)  # GeometryDataset
    push!(datasets, ds)

end

Boundary points for simplex 1:


In [35]:
typeof(Matrix(Diagonal([1.0, 1.0, -1.0, -1.0])))

Matrix{Float64} (alias for Array{Float64, 2})

In [ ]:

# ------------------------------------------------------------
# 5. 3D tetra data from 4D embedding
#    get3dtet(edgevec[i], solgso13[i]) → (3d_edges, zero_pos, sgndet)
# ------------------------------------------------------------
tet3d = [get3dtet(edges, solgso13[i]) for (i, edges) in enumerate(edgevec)]
threededgevec = [t[1] for t in tet3d]
sgndet        = [t[3] for t in tet3d]

In [5]:
geom = GeometryCollection(datasets);

In [6]:
# four_simplices = [[1,2,3,4,5], [1,2,4,5,6]];

# fix_kappa_signs!(four_simplices, geom);

# build_global_connectivity(four_simplices, geom);

In [7]:
# conn = build_global_connectivity(four_simplices, geom)
# push!(geom.connectivity, conn);

In [8]:
# run_face_xi_matching(geom);

In [9]:
# check_all(geom)

In [10]:
# run_su2_su11_gauge_fix(geom);

In [13]:
γ = 1.0
vars = run_define_variables(geom; gamma = γ);

In [20]:
using LinearAlgebra
using .SpinAlgebra: σ3

function halfedgeaction(xi, z, g, κ, meta; signzz=1, γ=1)
    # scalar contractions
    A = :( $signzz * conj($xi)' * $meta * transpose($g) * $z )
    B = :( $signzz * conj(transpose($g) * $z)' * $meta * $xi )

    term1 = :( 2 * log( ($A)^(( $κ + det($meta) ) / 2)
                        * ($B)^(( -$κ + det($meta) ) / 2) ) )

    term2 = :( (im * $γ * $κ - det($meta)) *
               log( $signzz * conj(transpose($g) * $z)' * $meta * transpose($g) * $z ) )

    return :( $term1 + $term2 )
end

function halfedgeactiont(xi, z, g, κ, meta; signzz=1, γ=1)
    A = :( $signzz * conj($xi)' * $meta * transpose($g) * $z )
    B = :( $signzz * conj(transpose($g) * $z)' * $meta * $xi )

    term1 = :( 2 * $κ * log( sqrt( $A / $B ) ) )
    term2 = :( -im / $γ * $κ * log( $A * $B ) )

    return :( $term1 + $term2 )
end


# ============================================================
# Vertex action (single 4-simplex)
# ============================================================

"""
    vertexaction(jfdata, xidata, zdata, gdata,
                 κdata, sgndet, tetn0sign, facesigndata;
                 γ=1)

Direct translation of Mathematica `vertexaction`.
All inputs are 5×5 or length-5 arrays, exactly as in DefineVariables.
"""
function vertexaction(jfdata, xidata, zdata, gdata,
                      κdata, sgndet, tetn0sign, facesigndata;
                      γ=1)

    ntet = 5
    act_terms = Expr[]

    for i in 1:ntet, j in 1:ntet
        i == j && continue

        jf = jfdata[i][j]
        jf == 0 && continue

        κ = κdata[i][j]

        # choose z orientation
        z = κ == 1 ? zdata[i][j] : zdata[j][i]

        if facesigndata[i][j] > 0
            meta = sgndet[i] == 1 ? I(2) : σ3
            signzz = tetn0sign[i][j]

            he = halfedgeaction(
                xidata[i][j], z, gdata[i],
                κ, meta;
                signzz=signzz, γ=γ
            )
        else
            he = halfedgeactiont(
                xidata[i][j], z, gdata[i],
                κ, σ3;
                signzz=1, γ=γ
            )
        end

        push!(act_terms, :( $jf * $he ))
    end

    return foldl((a,b)->:( $a + $b ), act_terms; init=:(0))
end

vertexaction

In [34]:
# unpack
jlabels = vars.jvariablesall[1]        # single simplex
bdyxi   = geom.simplex[1].bdyxi
zvars   = vars.zvariablesall[1]
gvars   = vars.gvariablesall[1]
xivars  = vars.xi_expr[1]
kappa   = geom.simplex[1].kappa
sgndet  = geom.simplex[1].sgndet
tetn0   = geom.simplex[1].tetn0sign
facesigndata = geom.simplex[1].tetareasign

5-element Vector{Vector{Int64}}:
 [0, 1, 1, 1, 1]
 [1, 0, 1, 1, 1]
 [1, 1, 0, -1, -1]
 [1, 1, -1, 0, -1]
 [1, 1, -1, -1, 0]

In [22]:

# symbolic vertex action
actv = vertexaction(
    jlabels,
    bdyxi,
    zvars,
    gvars,
    kappa,
    sgndet,
    tetn0,
    facesigndata;
    γ = 1
)

:((((((((((((((((((((0 + j1_1_2 * (2 * log((0 * (conj(Vector{ComplexF64}[[0.7071067811865477 + 0.0im, -0.7071067811865475 + 0.0im], [0.7071067811865475 + 0.0im, 0.7071067811865477 + 0.0im]]))' * Bool[1 0; 0 1] * transpose(Any[:(Ga1_1 + im * Gac1_1) :(Gb1_1 + im * Gbc1_1); :(Gc1_1 + im * Gcc1_1) :(Gd1_1 + im * Gdc1_1)]) * Any[1, :(z1_12 + im * zc1_12)]) ^ ((1 + det(Bool[1 0; 0 1])) / 2) * (0 * (conj(transpose(Any[:(Ga1_1 + im * Gac1_1) :(Gb1_1 + im * Gbc1_1); :(Gc1_1 + im * Gcc1_1) :(Gd1_1 + im * Gdc1_1)]) * Any[1, :(z1_12 + im * zc1_12)]))' * Bool[1 0; 0 1] * Vector{ComplexF64}[[0.7071067811865477 + 0.0im, -0.7071067811865475 + 0.0im], [0.7071067811865475 + 0.0im, 0.7071067811865477 + 0.0im]]) ^ ((-1 + det(Bool[1 0; 0 1])) / 2)) + (im * 1 * 1 - det(Bool[1 0; 0 1])) * log(0 * (conj(transpose(Any[:(Ga1_1 + im * Gac1_1) :(Gb1_1 + im * Gbc1_1); :(Gc1_1 + im * Gcc1_1) :(Gd1_1 + im * Gdc1_1)]) * Any[1, :(z1_12 + im * zc1_12)]))' * Bool[1 0; 0 1] * transpose(Any[:(Ga1_1 + im * Gac1_1) :(Gb1_1

In [25]:
crit = merge(
    vars.gsoln,
    vars.zsoln,
    vars.zeta_data,
    vars.jsoln
)

Dict{Symbol, Float64} with 96 entries:
  :zeta_1_1_2_b => 3.14159
  :j1_4_2       => 0.612372
  :gcc1_3       => 0.5
  :zeta_1_1_5_a => 0.0
  :zeta_1_3_1_b => 0.0
  :zc1_23       => 0.911438
  :zc1_53       => -0.707107
  :z1_31        => -0.707107
  :zc1_45       => 0.0
  :zeta_1_4_2_b => 0.0
  :zc1_12       => 0.0
  :zeta_1_3_5_b => 0.785398
  :gb1_2        => -0.278119
  :j1_4_5       => 0.25
  :zeta_1_2_5_a => 0.0
  :z1_12        => -1.0
  :zeta_1_5_1_b => 0.0
  :j1_2_5       => 0.433013
  :zeta_1_4_5_b => 0.0
  ⋮             => ⋮

In [32]:
using MacroTools: postwalk
function substitute_expr(ex, dict::Dict{Symbol,Float64})
    return Base.postwalk(ex) do x
        x isa Symbol && haskey(dict, x) ? dict[x] : x
    end
end

substitute_expr (generic function with 1 method)

In [33]:
crit_full = merge(crit, Dict(:γ => γ))   # γ must be numeric

actv_sub = substitute_expr(actv, crit_full)
actv_num = eval(actv_sub) |> complex

UndefVarError: UndefVarError: `postwalk` not defined in `Base`
Suggestion: check for spelling errors or missing imports.

:(2 * log((1 * (conj(Any[:(sin(zeta_1_1_2_a)), :(cos(zeta_1_1_2_a) * exp(im * zeta_1_1_2_b))]))' * ComplexF64[1.0 + 0.0im 0.0 + 0.0im; 0.0 + 0.0im 1.0 + 0.0im] * transpose(Any[:(Ga1_1 + im * Gac1_1) :(Gb1_1 + im * Gbc1_1); :(Gc1_1 + im * Gcc1_1) :(Gd1_1 + im * Gdc1_1)]) * Any[1, :(z1_12 + im * zc1_12)]) ^ ((1 + det(ComplexF64[1.0 + 0.0im 0.0 + 0.0im; 0.0 + 0.0im 1.0 + 0.0im])) / 2) * (1 * (conj(transpose(Any[:(Ga1_1 + im * Gac1_1) :(Gb1_1 + im * Gbc1_1); :(Gc1_1 + im * Gcc1_1) :(Gd1_1 + im * Gdc1_1)]) * Any[1, :(z1_12 + im * zc1_12)]))' * ComplexF64[1.0 + 0.0im 0.0 + 0.0im; 0.0 + 0.0im 1.0 + 0.0im] * Any[:(sin(zeta_1_1_2_a)), :(cos(zeta_1_1_2_a) * exp(im * zeta_1_1_2_b))]) ^ ((-1 + det(ComplexF64[1.0 + 0.0im 0.0 + 0.0im; 0.0 + 0.0im 1.0 + 0.0im])) / 2)) + (im * 1 * 1 - det(ComplexF64[1.0 + 0.0im 0.0 + 0.0im; 0.0 + 0.0im 1.0 + 0.0im])) * log(1 * (conj(transpose(Any[:(Ga1_1 + im * Gac1_1) :(Gb1_1 + im * Gbc1_1); :(Gc1_1 + im * Gcc1_1) :(Gd1_1 + im * Gdc1_1)]) * Any[1, :(z1_12 + im * zc1_

In [70]:
using Symbolics

function expr_to_symbolics(ex)
    if ex isa Number
        return ex                      # ← IMPORTANT: keep numbers as numbers
    elseif ex isa Symbol
        return Symbolics.variable(ex)
    elseif ex isa Expr
        args = map(expr_to_symbolics, ex.args)
        return Symbolics.Term(ex.head, args)
    elseif ex isa AbstractArray
        return map(expr_to_symbolics, ex)
    else
        return ex
    end
end

sexpr = expr_to_symbolics(expr)

subs = Dict(Symbolics.variable(k) => v for (k, v) in crit)

sexpr_sub = Symbolics.substitute(sexpr, subs)

val = Symbolics.evaluate(sexpr_sub)

MethodError: MethodError: objects of type Symbol are not callable
The object of type `Symbol` exists, but no method is defined for this combination of argument types when trying to treat it as a callable object.

In [76]:
function eval_with_values(expr::Expr, vals::Dict{Symbol,Float64})
    assigns = [:( $k = $(vals[k]) ) for k in keys(vals)]
    return eval(:(let $(assigns...); $expr; end))
end

eval_with_values (generic function with 2 methods)

In [77]:
val = eval_with_values(expr, crit)

MethodError: MethodError: no method matching *(::Int64, ::Expr)
The function `*` exists, but no method is defined for this combination of argument types.

Closest candidates are:
  *(::Any, ::Any, !Matched::Any, !Matched::Any...)
   @ Base operators.jl:596
  *(::Integer, !Matched::ForwardDiff.Dual{Ty}) where Ty
   @ ForwardDiff ~/.julia/packages/ForwardDiff/Wq9Wb/src/dual.jl:149
  *(::Number, !Matched::Num)
   @ Symbolics ~/.julia/packages/SymbolicUtils/KmZ71/src/methods.jl:76
  ...


In [78]:
function build_numeric_function(expr, syms)
    args = Expr(:tuple, syms...)
    fexpr = Expr(:function, args, expr)
    return eval(fexpr)
end

syms = collect(keys(crit))
f = build_numeric_function(expr, syms)
val = f(values(crit)...)

MethodError: MethodError: no method matching *(::Int64, ::Expr)
The function `*` exists, but no method is defined for this combination of argument types.

Closest candidates are:
  *(::Any, ::Any, !Matched::Any, !Matched::Any...)
   @ Base operators.jl:596
  *(::Integer, !Matched::ForwardDiff.Dual{Ty}) where Ty
   @ ForwardDiff ~/.julia/packages/ForwardDiff/Wq9Wb/src/dual.jl:149
  *(::Number, !Matched::Num)
   @ Symbolics ~/.julia/packages/SymbolicUtils/KmZ71/src/methods.jl:76
  ...
